In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.keras.datasets import mnist

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
%matplotlib inline

# Model

In [4]:
# Device configuration
device = torch.device('mps')

class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        # First convolution: 1 input channel (grayscale), 32 output channels,
        # kernel size of 3 with padding=1 to keep the input dimensions.
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Second convolution: 32 input channels, 64 output channels.
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # Max pooling layer to reduce spatial dimensions by a factor of 2.
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layers:
        # After two pooling layers, the 28x28 image becomes 7x7, and with 64 channels,
        # the flattened feature vector size is 64 * 7 * 7.
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # Input x shape: (batch_size, 1, 28, 28)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)  # Shape becomes: (batch_size, 32, 14, 14)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)  # Shape becomes: (batch_size, 64, 7, 7)
        
        # Flatten the tensor for the fully connected layers.
        x = x.view(x.size(0), -1)  # Shape: (batch_size, 64*7*7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Example: instantiate and send model to device
model = CNN(num_classes=10).to(device)

# Data

In [5]:
# 1. Load from Keras (already downloaded)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 2. Normalize and convert to tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Apply transform manually to each image
X_train = torch.stack([transform(img.astype(np.uint8)) for img in X_train])
X_test = torch.stack([transform(img.astype(np.uint8)) for img in X_test])

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# 3. Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training

In [10]:
# 2. Hyperparameters
input_size = 784  # 28x28
hidden_size = 128
num_classes = 10
num_epochs = 5
learning_rate = 0.001


model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# 5. Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# 6. Training Loop
for epoch in range(num_epochs):
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 0.2600
Epoch [2/5], Loss: 0.0182
Epoch [3/5], Loss: 0.0261
Epoch [4/5], Loss: 0.0690
Epoch [5/5], Loss: 0.1967


# Evaluation

In [12]:
# 7. Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10,000 test images: {100 * correct / total:.2f}%')

Accuracy of the model on the 10,000 test images: 97.61%
